## Setup

*You must run the cells in this section each time you connect to a new runtime. For example, when you return to the notebook after an idle timeout, when the runtime crashes, or when you restart or factory reset the runtime.*

Install requirements:


In [ ]:
! pip install --upgrade 'ocdskingfishercolab<0.4' ipywidgets plotly psycopg2-binary > pip.log

Import packages and load extensions:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from google.colab.data_table import DataTable
from google.colab.files import download
from ipywidgets import widgets
from ocdskingfishercolab import (
    download_dataframe_as_csv,
    format_thousands,
    render_json,
    save_dataframe_to_sheet,
    save_dataframe_to_spreadsheet,
    set_dark_mode,
    set_light_mode,
    set_spreadsheet_name,
)

# Load https://pypi.org/project/ipython-sql/
%load_ext sql
# Load https://colab.research.google.com/notebooks/data_table.ipynb
%load_ext google.colab.data_table

Set the name of the spreadsheet to export results to, when using `save_dataframe_to_sheet()`:

In [ ]:
spreadsheet_name = "feedback_results"

set_spreadsheet_name(spreadsheet_name)

Configure the notebook environment:

In [ ]:
# Increase max columns so that Pandas DataFrames with many columns are rendered as data tables.
DataTable.max_columns = 50
# Remove the index from data tables for easier copy-pasting to Google Docs.
DataTable.include_index = False

# Return Pandas DataFrames instead of regular result sets.
%config SqlMagic.autopandas = True
# Don't print number of rows affected.
%config SqlMagic.feedback = False

# If you set Tools > Settings > Site > Theme to dark, uncomment this line.
# set_dark_mode()
# If you are creating plots to copy-paste into reports, uncomment this line.
# set_light_mode()

## Setup field list

To run this notebook, you need to have the list of fields that the publisher is intended to publish. You can get that list from their mapping template. We currently don't have an automatic way to extract the list from the template, so you must get this list manually.

Upload an Excel file that includes the list of fields available in a column (no header).  Change the name of the file below. The list of fields must have the object/field_name format, for example `tender/id`.

In [ ]:
file = "ADD FILE"

Transform the file into a dataframe


In [ ]:
fields_table = pd.read_excel(file, header=None)
fields_table

Save fields table to a spreadsheet.  Save the spreadsheet in the relevant publisher folder.

In [ ]:
save_dataframe_to_sheet(fields_table, "fields")

## Usability analysis

Use this section to perform a usability analysis of the dataset, to identify if a publisher has the necessary fields to calculate 71 procurement indicators related to Market Opportunity (Market description, Competition, supplier performance), value for money, internal efficiency, public integrity and service delivery.  For OCDS publisher it also calculates the proportion of unique procedures for which it is possible to calculate the indicator (coverage).

The usability checks includes all the indicators listed on [OCP's use case guide](https://docs.google.com/spreadsheets/d/1j-Y0ktZiOyhZzi-2GSabBCnzx6fF5lv8h1KYwi_Q9GM/edit#gid=1183427361) and the [Indicators to diagnose the performance of a
procurement market document](https://docs.google.com/document/d/1vSJk9-qWSTQEx9ZZc7BUhQZMHvTRcyDYVS2sl8HB__k/edit#heading=h.nrnq1ajwwpqe).

### Usability setup

Use this section to set up the variables needed for the usability analysis.



Generate a list of the fields published:

In [ ]:
fields_list = fields_table.iloc[:, 0].tolist()

#### Set automatically which fields to use

To calculate some indicators there are alternative fields that can be used, for example to calculate the number of tenderers  both the `tender/numberOfTenderers` or the `tender/tenderers/id` could be used.  This section checks which fields are available in the publication.

In [ ]:
# U002
buyer = ["buyer/name", "buyer/id"]
procuring = ["tender/procuringEntity/name", "tender/procuringEntity/id"]
parties = ["parties/identifier/name", "parties/identifier/id", "parties/roles"]
if not any(item not in fields_list for item in buyer):
    buyer_var = buyer
elif not any(item not in fields_list for item in procuring):
    buyer_var = procuring
elif not any(item not in fields_list for item in parties):
    buyer_var = parties
else:
    buyer_var = buyer

# U003
if "tender/tenderers/id" in fields_list:
    bidders_val = "tender/tenderers/id"
elif "bids/details/tenderers/id" in fields_list:
    bidders_val = "bids/details/tenderers/id"
else:
    bidders_val = "tender/tenderers/id"

# U008
if "tender/items/classification/id" in fields_list and "tender/items/classification/scheme" in fields_list:
    items_val = ["tender/items/classification/id", "tender/items/classification/scheme"]
elif "awards/items/classification/id" in fields_list and "awards/items/classification/scheme" in fields_list:
    items_val = ["awards/items/classification/id", "awards/items/classification/scheme"]
elif "contracts/items/classification/id" in fields_list and "contractsitems/classification/scheme" in fields_list:
    items_val = ["contracts/items/classification/id", "contracts/items/classification/scheme"]
else:
    items_val = ["tender/items/classification/id", "tender/items/classification/scheme"]

# U012
if "contracts/id" in fields_list and "contracts/status" in fields_list:
    awards_val = ["contracts/id", "contracts/status"]
elif "awards/id" in fields_list and "awards/status" in fields_list:
    awards_val = ["awards/id", "awards/status"]
else:
    awards_val = ["contracts/id", "contracts/status"]

# U013, UC14
awards = ["awards/id", "awards/status", "awards/value/amount", "awards/value/currency"]
contracts = ["contracts/id", "contracts/status", "contracts/value/amount", "contracts/value/currency"]
if not any(item not in fields_list for item in contracts):
    awards_val2 = contracts
elif not any(item not in fields_list for item in awards):
    awards_val2 = awards
else:
    awards_val2 = contracts

# U015
if "tender/numberOfTenderers" in fields_list:
    bidders_val2 = "tender/numberOfTenderers"
elif "tender/tenderers/id" in fields_list:
    bidders_val2 = "tender/tenderers/id"
elif "bids/details/tenderers/id" in fields_list:
    bidders_val2 = "bids/details/tenderers/id"
else:
    bidders_val2 = "tender/numberOfTenderers"

# U034
aw = [
    "awards/status",
    "awards/value/amount",
    "awards/value/currency",
    "awards/items/classification/id",
    "awards/items/classification/scheme",
]
con = [
    "contracts/status",
    "contracts/value/amount",
    "contracts/value/currency",
    "contracts/items/classification/id",
    "contracts/items/classification/scheme",
]
if not any(item not in fields_list for item in aw):
    awards_val3 = aw
elif not any(item not in fields_list for item in con):
    awards_val3 = con
else:
    awards_val3 = aw

# U042
if "awards/status" in fields_list:
    awards_val4 = "awards/status"
elif "contracts/status" in fields_list:
    awards_val4 = "contracts/status"
else:
    awards_val4 = "awards/status"

# U061
if "contracts/period/startDate" in fields_list:
    contract_date = "contracts/period/startDate"
elif "awards/contractPeriod/startDate" in fields_list:
    contract_date = "awards/contractPeriod/startDate"
else:
    contract_date = "contracts/period/startDate"

# U065
aw2 = ["awards/items/quantity", "awards/items/unit"]
con2 = ["contracts/items/quantity", "contracts/items/unit"]
if not any(item not in fields_list for item in aw):
    awards_val5 = aw2
elif not any(item not in fields_list for item in con):
    awards_val5 = con2
else:
    awards_val5 = aw2

# U066, U067
if "planning/budget/amount/amount" in fields_list and "planning/budget/amount/currency":
    planning = ["planning/budget/amount/amount", "planning/budget/amount/currency"]
elif "tender/value/amount" in fields_list and "tender/value/currency":
    planning = ["tender/value/amount", "tender/value/currency"]
else:
    planning = ["planning/budget/amount/amount", "planning/budget/amount/currency"]

#### Dictionary of indicators

In [ ]:
indicators_dic = {
    "Total number of procedures": [["U001"], ["ocid"]],
    "Total number of procuring entities": [["U002"], ["ocid", *buyer_var]],
    "Total number of unique bidders": [["U003"], ["ocid", bidders_val]],
    "Total number of awarded suppliers": [
        ["U004"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/status"],
    ],
    "Total number of procedures by year or month": [["U005"], ["ocid", "date"]],
    "Total value awarded": [["U006"], ["ocid", "awards/status", "awards/value/amount", "awards/value/currency"]],
    "Share of procedures by status": [["U007"], ["ocid", "tender/status"]],
    "Number of procedures by item type": [["U008"], ["ocid", *items_val]],
    "Proportion of procedures by procurement category": [["U009"], ["ocid", "tender/mainProcurementCategory"]],
    "Percent of tenders by procedure type": [["U010"], ["ocid", "tender/procurementMethod"]],
    "Percent of tenders awarded by means of competitive procedures": [
        ["U011"],
        ["ocid", "tender/procurementMethod", "awards/status"],
    ],
    "Percent of contracts awarded under each procedure type": [
        ["U012"],
        ["ocid", "tender/procurementMethod", *awards_val],
    ],
    "Total contracted value awarded under each procedure type": [
        ["U013"],
        ["ocid", "tender/procurementMethod", *awards_val2],
    ],
    "Total awarded value of tenders awarded by means of competitive procedures": [
        ["U014"],
        ["ocid", "tender/procurementMethod", *awards_val2],
    ],
    "Proportion of single bid tenders": [["U015"], ["ocid", "tender/procurementMethod", bidders_val2]],
    "Proportion of value awarded in single bid tenders vs competitive tenders": [
        ["U016"],
        [
            "ocid",
            "tender/procurementMethod",
            "awards/status",
            "awards/value/amount",
            "awards/value/currency",
            bidders_val2,
        ],
    ],
    "Mean number of bidders per tender": [["U017"], ["ocid", "tender/procurementMethod", bidders_val2]],
    "Median number of bidders per tender": [["U018"], ["ocid", "tender/procurementMethod", bidders_val2]],
    "Mean number of bidders by item type": [["U019"], ["ocid", "tender/procurementMethod", bidders_val2, *items_val]],
    "Number of suppliers by item type": [
        ["U020"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *items_val],
    ],
    "Number of new bidders in a system ": [
        ["U021"],
        ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"],
    ],
    "Percent of new bidders to all bidders ": [
        ["U022"],
        ["tender/id", "tender/tenderers/id", "tender/tenderPeriod/startDate"],
    ],
    "Percent of tenders with at least three participants deemed qualified": [
        ["U023"],
        ["ocid", "bids/details/tenderers/id", "bids/details/id", "bids/details/status"],
    ],
    "Mean percent of bids which are disqualified": [["U024"], ["tender/id", "bids/details/id", "bids/details/status"]],
    "Percent of contracts awarded to top 10 suppliers with largest contracted totals": [
        ["U025"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *awards_val2],
    ],
    "Mean number of unique suppliers per buyer": [
        ["U026"],
        ["ocid", "awards/suppliers/id", "awards/suppliers/name", *buyer_var],
    ],
    "Number of new awarded suppliers ": [
        ["U027"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
    ],
    "Percent of awards awarded to new suppliers": [
        ["U028"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
    ],
    "Total awarded value awarded to new suppliers": [
        ["U029"],
        [
            "awards/id",
            "awards/suppliers/id",
            "awards/suppliers/name",
            "awards/date",
            "awards/value/amount",
            "awards/value/currency",
        ],
    ],
    "Percent of new suppliers to all suppliers": [
        ["U030"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
    ],
    "Percent of growth of new awarded suppliers in a system": [
        ["U031"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", "awards/date"],
    ],
    "Percent of total awarded value awarded to recurring suppliers": [
        ["U032"],
        [
            "awards/id",
            "awards/suppliers/id",
            "awards/suppliers/name",
            "awards/date",
            "awards/value/amount",
            "awards/value/currency",
        ],
    ],
    "Mean number of bids necessary to win": [
        ["U033"],
        ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"],
    ],
    "Market concentration, market share of the largest company in the market": [
        ["U034"],
        ["awards/suppliers/id", "awards/suppliers/name", *awards_val3],
    ],
    "Proportion of contracts awarded by supplier by non competitive procedures": [
        ["U035"],
        ["ocid", "tender/procurementMethod", "awards/status", "awards/suppliers/id", "awards/suppliers/name"],
    ],
    "Region of the supplier": [["U036"], ["parties/roles", "parties/identifier/id", "parties/address/region"]],
    "Number of bids submitted by supplier": [["U037"], ["awards/suppliers/id", bidders_val]],
    "Success rate of bidders": [
        ["U038"],
        ["ocid", "tender/tenderers/id", "awards/suppliers/id", "awards/suppliers/name"],
    ],
    "Number of unique items classifications awarded by supplier": [
        ["U039"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", *items_val],
    ],
    "Total value awarded by supplier": [["U040"], ["awards/suppliers/id", "awards/suppliers/name", *awards_val2]],
    "Share of total value awarded by supplier": [
        ["U041"],
        ["awards/suppliers/id", "awards/suppliers/name", *awards_val2],
    ],
    "Total number of contracts awarded by supplier": [
        ["U042"],
        ["awards/id", "awards/suppliers/id", "awards/suppliers/name", awards_val4],
    ],
    "Number of procuring entities by supplier": [
        ["U043"],
        ["ocid", "awards/suppliers/id", "awards/suppliers/name", *buyer_var],
    ],
    "Share of single bid awards by supplier": [
        ["U044"],
        [
            "ocid",
            "awards/suppliers/id",
            "awards/suppliers/name",
            "awards/status",
            "tender/procurementMethod",
            bidders_val2,
        ],
    ],
    "Percent of tenders with linked procurement plans": [["U045"], ["tender/id", "tender/documents/documentType"]],
    "Percent of contracts which publish information on debarments": [
        ["U046"],
        ["contracts/id", "contracts/implementation/documents/documentType"],
    ],
    "The percent of tenders for which the tender documentation was added after publication of the announcement ": [
        ["U047"],
        [
            "tender/id",
            "tender/documents/documentType",
            "tender/documents/documentType",
            "tender/documents/datePublished",
        ],
    ],
    "Mean number of contract amendments per buyer": [
        ["U048"],
        ["ocid", "contracts/id", "contracts/amendments", *buyer_var],
    ],
    "Percent of tenders which have been closed for more than 30 days, but whose basic awards information is not published": [
        ["U049"],
        [
            "tender/id",
            "tender/tenderPeriod/endDate",
            "awards/id",
            "awards/date",
            "awards/status",
            "awards/value/amount",
            "awards/suppliers/id",
            "awards/suppliers/name",
        ],
    ],
    "Percent of awards which are older than 30 days, but whose contract is not published": [
        ["U050"],
        [
            "awards/id",
            "awards/date",
            "contracts/awardID",
            "contracts/status",
            "contracts/dateSigned",
            "contracts/documents/documentType",
        ],
    ],
    "Percent of tenders that do not specify place of delivery": [
        ["U051"],
        ["ocid", "tender/items/deliveryLocation", "tender/items/deliveryAddress"],
    ],
    "Percent of tenders that do not specify date of delivery": [
        ["U052"],
        [
            "tender/milestones/id",
            "tender/milestones/type",
            "tender/milestones/description",
            "tender/milestones/dueDate",
        ],
    ],
    "Percent of tenders with short titles for example fewer than 10 characters in the title": [
        ["U053"],
        ["tender/id", "tender/title"],
    ],
    "Percent of tenders with short descriptions for instance fewer than 30 characters in the description": [
        ["U054"],
        ["tender/id", "tender/description"],
    ],
    "Percent of tenders that do not include detailed item codes or item descriptions": [
        ["U055"],
        ["tender/id", "tender/items/classification/id", "tender/items/classification/scheme"],
    ],
    "Percent of contracts that do not have amendments": [["U056"], ["contracts/id", "contracts/amendments"]],
    "Percent of contracts which publish contract implementation details financial": [
        ["U057"],
        [
            "contracts/implementation/transactions/id",
            "contracts/implementation/transactions/value/amount",
            "contracts/implementation/transactions/value/currency",
        ],
    ],
    "Percent of contracts which publish contract implementation details physical": [
        ["U058"],
        [
            "contracts/implementation/milestones/type",
            "contracts/implementation/milestones/id",
            "contracts/implementation/milestones/dueDate",
            "contracts/implementation/milestones/status",
        ],
    ],
    "Average duration of tendering period days": [
        ["U059"],
        ["ocid", "tender/tenderPeriod/startDate", "tender/tenderPeriod/endDate"],
    ],
    "Average duration of decision period days": [["U060"], ["ocid", "tender/tenderPeriod/endDate", "awards/date"]],
    "Average days from award date to start of implementation": [["U061"], ["awards/id", "awards/date", contract_date]],
    "Days between award date and tender start date": [
        ["U062"],
        ["ocid", "tender/tenderPeriod/startDate", "awards/date"],
    ],
    "Percent of canceled tenders to awarded tenders": [["U063"], ["ocid", "tender/status", "awards/status"]],
    "Percent of contracts which are canceled": [["U064"], ["contracts/id", "contracts/status"]],
    "Price variation of same item across all awards": [["U065"], awards_val3 + awards_val5],
    "Percent of contracts that exceed budget": [
        ["U066"],
        ["ocid", "contracts/status", "contracts/value/amount", "contracts/value/currency", *planning],
    ],
    "Mean percent overrun of contracts that exceed budget": [
        ["U067"],
        ["ocid", "contracts/status", "contracts/value/amount", "contracts/value/currency", *planning],
    ],
    "Total percent savings difference between budget and contract value": [
        ["U068"],
        [
            "ocid",
            "planning/budget/amount/amount",
            "planning/budget/amount/currency",
            "contracts/value/amount",
            "contracts/value/currency",
        ],
    ],
    "Total percent savings difference between tender value estimate and contract value": [
        ["U069"],
        ["ocid", "tender/value/amount", "tender/value/currency", "contracts/value/amount", "contracts/value/currency"],
    ],
    "Percent of contracts completed on time ": [
        ["U070"],
        ["contracts/id", "contracts/period/endDate", "contracts/status"],
    ],
    "Share of contracts whose milestones are completed on time": [
        ["U071"],
        ["contracts/id", "contracts/implementation/milestones/dueDate", "contracts/implementation/milestones/dateMet"],
    ],
}

#### Usability function

In [ ]:
from ocdskingfishercolab import calculate_coverage

In [ ]:
def usability_checks(check_coverage=False):
    """Returns a table of the usability checks.
    It indicates if the fields needed to calculate a particular indicator are present.
    Set check_coverage=True to check for coverage"""

    results_list = []
    missing_fields = []

    for i in indicators_dic.values():
        check = any(item not in fields_list for item in i[1])
        if check:
            result = "missing fields"

        else:
            result = "possible to calculate"
        missing = [i[1][j] for j in range(0, len(i[1])) if i[1][j] not in fields_list]
        missing_fields.append(missing)
        results_list.append(result)

    # Generate dataframe

    df = pd.DataFrame(
        list(
            zip(
                list(indicators_dic.keys()),
                [indicators_dic[i][0] for i in indicators_dic.keys()],
                [indicators_dic[i][1:] for i in indicators_dic.keys()],
            )
        ),
        columns=["indicator", "U_id", "fields needed"],
    )
    df["U_id"] = df["U_id"].apply(lambda x: ", ".join(map(str, x)))
    df["fields needed"] = df["fields needed"].astype(str).str.replace("\[|\]|'", "")
    df["calculation"] = results_list
    df["missing fields"] = missing_fields
    df["missing fields"] = df["missing fields"].apply(lambda x: ", ".join(map(str, x)))

    if check_coverage:
        # Calculate coverage
        coverage = []
        for i in indicators_dic.values():
            fields = [item for sublist in i for item in sublist][1:]
            result = calculate_coverage(fields, "release_summary")
            result_value = pd.to_numeric(result["total_percentage"][0])
            coverage.append(result_value)
        df["coverage"] = coverage
        return df

    else:
        return df

Set `check_coverage=True` if you are working with an OCDS dataset and want to calculate coverage.  Default is set to `False`.

In [ ]:
result = usability_checks(check_coverage=True)

### Export and visualize results

#### Choose language of the export

In [ ]:
from ipywidgets import widgets

style = {"description_width": "initial"}


def pick_language():
    global lang
    languages = ["Spanish", "English"]
    lang = widgets.Dropdown(options=languages, description="language", style=style)
    display(lang)


pick_language()

#### Load use case indicators spreadsheet

In [ ]:
if lang.value == "English":
    language = "Use case guide: Indicators linked to OCDS #public"
else:
    language = "[ES] of Use case guide: Indicators linked to OCDS #public"

from google.colab import auth

auth.authenticate_user()

from google.auth import default

creds, _ = default()

import gspread

gc = gspread.authorize(creds)
worksheet = gc.open(language).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# Convert to a DataFrame and render.

indicators = pd.DataFrame(rows)
indicators = indicators.rename(columns=indicators.iloc[0]).drop(indicators.index[0])

if lang.value == "English":
    indicatorsdf = indicators.iloc[:, [0, 3, 4, 9]]
    result_final = result.merge(indicatorsdf, on="U_id")
else:
    indicatorsdf = indicators.iloc[:, [0, 3, 4, 5, 9]]
    result_final = indicatorsdf.merge(result, on="U_id").drop(["indicator"], axis=1)
    result_final.rename(
        columns={
            "fields needed": "Campos necesarios",
            "calculation": "¿Se puede calcular?",
            "missing fields": "Campos faltantes",
            "coverage": "Cobertura",
        },
        inplace=True,
    )
    result_final.replace(
        {"¿Se puede calcular?": {"possible to calculate": "sí", "missing fields": "campos faltantes"}}, inplace=True
    )

#### Table of results

In [ ]:
result_final

#### Most common fields for indicators

This table shows the most frequent fields used to calculate indicators and if they are published.  You can use this table to highlight to the publisher the key data gaps.  

In [ ]:
from collections import Counter

import numpy as np

fields = list(indicators_dic.values())
fields = [item[1:] for item in fields]
flat_list = [item for sublist in [item for sublist in fields for item in sublist] for item in sublist]
fields_list = Counter(flat_list)


fields_count = (
    pd.DataFrame.from_dict(fields_list, orient="index")
    .reset_index()
    .rename(columns={"index": "field", 0: "number of indicators"})
)


fields_count = fields_count.sort_values("number of indicators", ascending=False).reset_index(drop=True)
fields_count["published"] = np.where(fields_count["field"].isin(fields_table["path"]), "yes", "no")

fields_count

#### Save tables to spreadsheet

In [ ]:
save_dataframe_to_sheet(result_final, "usability table")
save_dataframe_to_sheet(fields_count, "key fields")

#### Visualize results

In [ ]:
import altair as alt

if lang.value == "English":
    alt.Chart(result_final).transform_window(
        nrow="row_number(indicator)", frame=[None, None], sort=[{"field": "calculation"}], groupby=["Use case"]
    ).mark_circle(size=250, opacity=1).encode(
        x=alt.X(
            "nrow", type="quantitative", axis=alt.Axis(title=["number of indicators", ""], orient="top", tickCount=5)
        ),
        y=alt.Y("Use case", type="nominal", sort=alt.Sort(field="indicator", op="count", order="descending")),
        color=alt.Color(
            "calculation",
            type="nominal",
            scale=alt.Scale(range=["#fb6045", "#d6e100"]),
            legend=alt.Legend(title=["can we calculate it?"]),
        ),
        tooltip=[
            alt.Tooltip("indicator", type="nominal"),
            alt.Tooltip("Use case", type="nominal"),
            alt.Tooltip("calculation", type="nominal"),
            alt.Tooltip("missing fields", type="nominal"),
        ],
    ).properties(
        width=600, height=350, padding=50, title=alt.TitleParams(text="", subtitle=[""], fontSize=18)
    ).configure_axis(
        titleFontSize=14, labelFontSize=14, labelPadding=5, ticks=False, domain=False
    ).configure_legend(
        labelFontSize=14, titleFontSize=14
    ).configure_view(
        strokeWidth=0
    ).display()

else:
    alt.Chart(result_final).transform_window(
        nrow="row_number(Indicador)",
        frame=[None, None],
        sort=[{"field": "¿Se puede calcular?"}],
        groupby=["Caso de Uso"],
    ).mark_circle(size=250, opacity=1).encode(
        x=alt.X(
            "nrow",
            type="quantitative",
            axis=alt.Axis(title=["Cantidad de indicadores", ""], orient="top", tickCount=5),
        ),
        y=alt.Y("Caso de Uso", type="nominal", sort=alt.Sort(field="Indicador", op="count", order="descending")),
        color=alt.Color(
            "¿Se puede calcular?",
            type="nominal",
            scale=alt.Scale(range=["#fb6045", "#d6e100"]),
            legend=alt.Legend(title=["¿Se puede calcular?"]),
        ),
        tooltip=[
            alt.Tooltip("Indicador", type="nominal"),
            alt.Tooltip("Caso de uso", type="nominal"),
            alt.Tooltip("¿Se puede calcular?", type="nominal"),
            alt.Tooltip("Campos faltantes", type="nominal"),
        ],
    ).properties(
        width=600, height=350, padding=50, title=alt.TitleParams(text="", subtitle=[""], fontSize=18)
    ).configure_axis(
        titleFontSize=14, labelFontSize=14, labelPadding=5, ticks=False, domain=False
    ).configure_legend(
        labelFontSize=14, titleFontSize=14
    ).configure_view(
        strokeWidth=0
    ).display()